# The Battle of Neighborhoods - Hospitals in Italy

## Introduction

During the Covid-19 epidemic an important aspect that many people might consider when choosing a city to live in is the number of hospitals. In the event of a major worsening of the epidemic, a better equipped city as a number of places in intensive care could prove essential to better manage the situation and ensure short lockdown periods.

In this project, I am going to take five Italian cities of more or less the same size and check which is the best in case of a worsening of the epidemic.

All data regarding the number and location of hospitals in the city will be taken from the Foursquare API

## Data section

I will use the FourSquare API to collect data about locations of Pizza stores in five Italian cities which are: Bologna, Brescia, Palermo, Parma, Modena. 

## Methodology

In the first part of this project I will download from Foursquare API the data about the hospitals in the selected cities and put them on a map. Afterwards I will rank the number of hospitals per city, but this will not be the final goal since what really interests me is not the number of hospitals in a city but the density of hospitals in the city. So, in the second part, I am going to study this density, removing one outlier per city to be sure of the result.

### Let's first import libraries and connect our account 

In [1]:
! pip install folium

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'FTWZ44EL0QWFHTYPCEG2XL2CWWNUFISYOGWVA3WZODHELWVZ' # your Foursquare ID
CLIENT_SECRET = 'SGK2AXY230QOHWUCGWFO4TWSLQZJ2UWCXVDITOQPVZYM4LV2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## Step 1: let's check the number of hospitals for the four cities selected in Italy:

In [3]:
LIMIT = 100
cities = ["BOLOGNA, IT", 'MODENA, IT', 'BRESCIA, IT', 'PARMA, IT', 'PALERMO, IT']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d196941735") # HOSPITALS CATEGORY ID
    results[city] = requests.get(url).json()
    
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=14)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of hospitals in {city} = ", results[city]['response']['totalResults'])

Total number of hospitals in BOLOGNA, IT =  49
Total number of hospitals in MODENA, IT =  9
Total number of hospitals in BRESCIA, IT =  13
Total number of hospitals in PARMA, IT =  14
Total number of hospitals in PALERMO, IT =  21


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


### Map of Bologna

In [4]:
maps[cities[0]]

### Map of Modena

In [5]:
maps[cities[1]]

### Map of Brescia

In [6]:
maps[cities[2]]

### Map of Parma

In [7]:
maps[cities[3]]

### Map of Palermo

In [8]:
maps[cities[4]]

## Step 2: Calculate the density of hospitals per city

Since the absolute number of hospitals could be misleading as a given, since a larger city will certainly have a higher number of hospitals, I will try to analyse which city is the most densely "inhabited" of hospitals.

For this I will use some basic statistics. I will get the mean location of the hospitals which should be near to most of them if they are really dense or far if not. Next I will take the average of the distance of the venues to the mean coordinates

So, in the next phase we calculate the mean coordinate and the mean distance to mean coordinate (MDMC). We represent the mean coordinate with a big green circle and distances with green lines

In [9]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

BOLOGNA, IT
Mean Distance from Mean coordinates
0.01875795948177954
MODENA, IT
Mean Distance from Mean coordinates
0.015518644104469804
BRESCIA, IT
Mean Distance from Mean coordinates
0.019107103207395882
PARMA, IT
Mean Distance from Mean coordinates
0.011401407791817373
PALERMO, IT
Mean Distance from Mean coordinates
0.023870062069573164


In [10]:
maps[cities[0]]

In [11]:
maps[cities[1]]

In [12]:
maps[cities[2]]

In [13]:
maps[cities[3]]

In [14]:
maps[cities[4]]

## Discussion:

One thing that can be noticed is the presence of hospitals on the farthest outskirts of the city. This could alter the results. So let's try to remove this possible outlier

At the moment, from the results obtained we can see that the most "densely populated" city in terms of hospitals is Parma.

In [15]:
for city in cities:
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    print(city)
    print("Mean Distance from Mean coordinates")
    dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
    dists.sort()
    print(np.mean(dists[:-1]))# Ignore the biggest distance

BOLOGNA, IT
Mean Distance from Mean coordinates
0.017322994574057938
MODENA, IT
Mean Distance from Mean coordinates
0.011707403648582741
BRESCIA, IT
Mean Distance from Mean coordinates
0.016879764146019927
PARMA, IT
Mean Distance from Mean coordinates
0.010014489112195907
PALERMO, IT
Mean Distance from Mean coordinates
0.022188740312651667


## Conclusions:

We can note that, despite the removal of a possible outlier from the results and despite the fact that the ranking has changed slightly, the city that can still be relied on for the presence of hospitals is still Parma.